In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
# hyperparameters
EPOCHS = 40
BATCH_SIZE = 64

### 학습용과 테스트용 데이터셋 불러오기

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('C:/Users/gun84/data/FashionMNIST',
                           train = True,
                           transform = transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
    batch_size = BATCH_SIZE, shuffle = True)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('C:/Users/gun84/data/FashionMNIST',
                         train = False,
                         transform = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081,))
                         ])),
    batch_size = BATCH_SIZE, shuffle = True)

print(len(train_loader.dataset))

60000


#### CNN 모델 

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10,  5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.drop = nn.Dropout2d()
        
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = x.view(-1, 320)
        
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x

In [6]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

### train

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.
                 format(epoch, batch_idx * len(data),
                       len(train_loader.dataset),
                       100 * batch_idx / len(train_loader),
                       loss.item()))

### test

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            
            # 가장 높은 값을 가진 인텍스가 예측값이다
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
            epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss:2.292178
Train Epoch: 1 [12800/60000 (21%)]	Loss:1.074987
Train Epoch: 1 [25600/60000 (43%)]	Loss:1.225196
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.688239
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.773013
[1] Test Loss: 0.6660, Accuracy: 73.78%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.771110
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.602140
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.808639
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.809213
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.697859
[2] Test Loss: 0.5469, Accuracy: 78.78%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.623036
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.638745
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.609624
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.454645
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.635118
[3] Test Loss: 0.5014, Accuracy: 80.38%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.675837
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.641233
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.638504
Train Epoch: 4 [38

[29] Test Loss: 0.2995, Accuracy: 89.15%
Train Epoch: 30 [0/60000 (0%)]	Loss:0.297015
Train Epoch: 30 [12800/60000 (21%)]	Loss:0.271208
Train Epoch: 30 [25600/60000 (43%)]	Loss:0.421082
Train Epoch: 30 [38400/60000 (64%)]	Loss:0.507998
Train Epoch: 30 [51200/60000 (85%)]	Loss:0.388286
[30] Test Loss: 0.2989, Accuracy: 89.04%
Train Epoch: 31 [0/60000 (0%)]	Loss:0.356925
Train Epoch: 31 [12800/60000 (21%)]	Loss:0.209424
Train Epoch: 31 [25600/60000 (43%)]	Loss:0.289228
Train Epoch: 31 [38400/60000 (64%)]	Loss:0.326308
Train Epoch: 31 [51200/60000 (85%)]	Loss:0.204099
[31] Test Loss: 0.2994, Accuracy: 89.28%
Train Epoch: 32 [0/60000 (0%)]	Loss:0.349366
Train Epoch: 32 [12800/60000 (21%)]	Loss:0.276951
Train Epoch: 32 [25600/60000 (43%)]	Loss:0.292768
Train Epoch: 32 [38400/60000 (64%)]	Loss:0.322713
Train Epoch: 32 [51200/60000 (85%)]	Loss:0.259983
[32] Test Loss: 0.2972, Accuracy: 89.17%
Train Epoch: 33 [0/60000 (0%)]	Loss:0.356298
Train Epoch: 33 [12800/60000 (21%)]	Loss:0.410554
Train 

## ResNet

In [9]:
# 컬러 데이터셋은 흑백 이미지보다 복잡하므로 학습을 더 많이 해야한다.
EPOCHS = 300
BATCH_SIZE = 218

#### 데이터셋 불러오기

In [10]:
train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('C:/Users/gun84/data',
                        train = True,
                        download = True,
                        transform = transforms.Compose([
                            transforms.RandomCrop(32, padding = 4), # 이미지를 랜덤으로 아무대나 잘라 32 크기로 출력, 과적합 방지
                            transforms.RandomHorizontalFlip(),      # 과적합 방지
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5),
                                                (0.5, 0.5, 0.5))
                        ])),
        batch_size = BATCH_SIZE, shuffle = True)
test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('C:/Users/gun84/data',
                        train = False,
                        transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5),
                                                (0.5, 0.5, 0.5))
                        ])),
        batch_size = BATCH_SIZE, shuffle = True)

Files already downloaded and verified


#### Resdual 블록

In [11]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3,
                              stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3,
                              stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                         kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(planes))
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [13]:
class ResNet(nn.Module):
    def __init__(self,num_classes = 10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride = 1)
        self.layer2 = self._make_layer(32, 2, stride = 2)
        self.layer3 = self._make_layer(64, 2, stride = 2)
        self.linear = nn.Linear(64, num_classes)
    
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks - 1) # layer1 일때 strides = [1. 1]
        layers = []
        for stride in strides: # layer1: [1, 1], layer2: [2, 1], layer3: [2, 1]
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers) # 편하게 순차적으로 실행하도록 담는 container 라고 생각
    
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x))) # out.shape : ([1,16,32,32])
        out = self.layer1(out) # out.shape : ([1,16,32,32])
        out = self.layer2(out) 
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
#         print('out',out.shape)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
# img = torch.randn([128,3,32,32])
# a = ResNet()
# a(img)


In [14]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.1,
                     momentum = 0.9, weight_decay = 0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 50, gamma = 0.1)

In [15]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [16]:
for epoch in range(1, EPOCHS +1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

C:\Users\gun84\Anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 1 [0/50000 (0%)]	Loss:2.366076
Train Epoch: 1 [43600/50000 (87%)]	Loss:1.379607


NameError: name 'test' is not defined